In [10]:
import re

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import polars as pl
import polars.selectors as cs

In [11]:
LUD_VAR_RE = r'"[\w\s\(\)\'\/\.\,\:^\-]+"'

@pl.api.register_expr_namespace("strip_lud_rule")
class StripLudRule:
    __vars_dict = {}

    def __init__(self, expr: pl.Expr) -> None:
        self._expr = expr
    
    @classmethod
    def __choose_var_num(var_match: re.Match) -> str:
        var_text = var_match.group(0)
        var_ind = StripLudRule.__vars_dict.setdefault(var_text, None)
        if var_ind is None:
            var_ind = len(StripLudRule.__vars_dict) + 1
            StripLudRule.__vars_dict[var_text] = var_ind
        return f"VarName{var_ind}"

    def __call__(self) -> pl.Object:
        char_num = self._expr
        raw = self._expr.row()
        re.sub(LUD_VAR_RE, self.__choose_var_num, raw)
        StripLudRule.__vars_dict = {}
        return pl.lit(raw).cast(pl.Object)


C:\Users\ДТ\AppData\Local\Temp\ipykernel_12084\1902878792.py:3: UserWarning: Overriding existing custom namespace 'strip_lud_rule' (on 'Expr')
  @pl.api.register_expr_namespace("strip_lud_rule")


In [12]:
# Credits goes to yunsuxiaozi
# https://www.kaggle.com/code/yunsuxiaozi/mcts-deal-with-ludrules
# Slightly edited to be runable on polars dfs

def symbol_match(rule):
    stack=[]
    data=[]
    for i in range(len(rule)):
        if rule[i] in ['(','{']:
            stack.append(rule[i])
        elif rule[i] in [')','}']:
            stack=stack[:-1]
        elif (rule[i]=='"') and (len(stack)>0) and (stack[-1]=='"'):
            stack=stack[:-1]
        elif rule[i]=='"':
            stack.append('"')
        data.append(rule[i])
        if len(stack)==0:
            return ''.join(data).strip(),rule[i+1:].strip()
    return '',''
    
def get_ruledata(rule):
    rule=rule[len('(game '):-1].strip()
    datas=[]
    while len(rule):
        data,rule=symbol_match(rule)
        datas.append(data)
    return datas

In [13]:
df_pl = pl.read_csv('train_tuned.csv')

In [14]:
df_pl.head()

Id,GameRulesetName,agent1,agent2,Stochastic,Asymmetric,AsymmetricForces,AsymmetricPiecesType,PlayersWithDirections,Cooperation,Team,Shape,SquareShape,HexShape,TriangleShape,DiamondShape,RectangleShape,SpiralShape,CircleShape,StarShape,SquarePyramidalShape,RegularShape,PolygonShape,Tiling,SquareTiling,HexTiling,TriangleTiling,SemiRegularTiling,MorrisTiling,CircleTiling,ConcentricTiling,SpiralTiling,AlquerqueTiling,MancalaBoard,MancalaStores,MancalaTwoRows,MancalaThreeRows,…,CheckersComponent,BallComponent,TaflComponent,DiscComponent,MarkerComponent,StackType,Stack,Symbols,ShowPieceValue,ShowPieceState,Implementation,State,StateType,StackState,PieceState,SiteState,SetSiteState,VisitedSites,Variable,SetVar,RememberValues,ForgetValues,SetPending,InternalCounter,SetInternalCounter,PlayerValue,Efficiency,CopyContext,Then,ForEachPiece,DoLudeme,Trigger,PlayoutsPerSecond,MovesPerSecond,EnglishRules,LudRules,utility_agent1
i64,str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,…,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,str,str,f64
0,"""00Y""","""MCTS-ProgressiveHistory-0.1-MA…","""MCTS-ProgressiveHistory-0.6-Ra…",0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,…,1,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,1,0,0,1,298.07,18877.17,"""Goal: Connect all three edge c…","""(game ""00'Y'"" (players 2) (equ…",-0.466667
1,"""00Y""","""MCTS-ProgressiveHistory-0.1-MA…","""MCTS-UCB1GRAVE-0.6-NST-true""",0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,…,1,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,1,0,0,1,298.07,18877.17,"""Goal: Connect all three edge c…","""(game ""00'Y'"" (players 2) (equ…",-0.333333
2,"""00Y""","""MCTS-ProgressiveHistory-0.1-MA…","""MCTS-UCB1-0.1-NST-false""",0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,…,1,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,1,0,0,1,298.07,18877.17,"""Goal: Connect all three edge c…","""(game ""00'Y'"" (players 2) (equ…",-0.066667
3,"""00Y""","""MCTS-ProgressiveHistory-0.1-MA…","""MCTS-UCB1-0.6-NST-false""",0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,…,1,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,1,0,0,1,298.07,18877.17,"""Goal: Connect all three edge c…","""(game ""00'Y'"" (players 2) (equ…",-0.333333
4,"""00Y""","""MCTS-ProgressiveHistory-0.1-MA…","""MCTS-UCB1GRAVE-1.41421356237-N…",0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,…,1,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,1,0,0,1,298.07,18877.17,"""Goal: Connect all three edge c…","""(game ""00'Y'"" (players 2) (equ…",-0.333333


In [15]:
concepts_df = pl.read_csv('data/concepts.csv')

In [16]:
lud_rules = df_pl.select('LudRules').unique()

In [17]:
lud_rules

LudRules
str
"""(game ""Claustro"" (players 2) (…"
"""(game ""Mraha wa Bwe"" (players …"
"""(game ""Ludus Coriovalli"" (play…"
"""(game ""Connect6"" (players 2) (…"
"""(game ""Tenjiku Shogi"" (players…"
…
"""(game ""58 Holes"" (players 2) (…"
"""(game ""Ludus Coriovalli"" (play…"
"""(game ""Okwe (Nigeria)"" (player…"


In [18]:
# lud_rules.with_columns(
#     (pl.col('LudRules').strip_lud_rule().alias('striped')),
# )

In [19]:
boards_list = df_pl.select(cs.contains('Board')).columns

In [20]:
tracks_list = df_pl.select(cs.contains('Track')).columns

In [21]:
shapes_list = df_pl.select(cs.contains('Shape')).columns

In [22]:
tiling_list = df_pl.select(cs.contains('Tiling')).columns

In [23]:
drop_list = boards_list + tracks_list + shapes_list + tiling_list

In [24]:
df_stripped = df_pl.drop(drop_list)

In [25]:
other_categoricals = []

for column in df_stripped.columns:
    if df_pl.select(column).unique().count().to_numpy()[0][0] == 2:
        other_categoricals.append(column)

other_categoricals

['Stochastic',
 'Asymmetric',
 'AsymmetricForces',
 'AsymmetricPiecesType',
 'PlayersWithDirections',
 'Cooperation',
 'Team',
 'MancalaStores',
 'MancalaTwoRows',
 'MancalaThreeRows',
 'MancalaFourRows',
 'MancalaSixRows',
 'MancalaCircular',
 'Region',
 'Vertex',
 'Cell',
 'Edge',
 'Hand',
 'Piece',
 'PieceValue',
 'PieceRotation',
 'PieceDirection',
 'Dice',
 'DiceD2',
 'DiceD4',
 'DiceD6',
 'LargePiece',
 'Tile',
 'Meta',
 'OpeningContract',
 'SwapOption',
 'Repetition',
 'TurnKo',
 'PositionalSuperko',
 'AutoMove',
 'Start',
 'InitialRandomPlacement',
 'InitialScore',
 'InitialCost',
 'Moves',
 'MovesDecision',
 'NoSiteMoves',
 'VoteDecision',
 'SwapPlayersDecision',
 'SwapPlayersDecisionFrequency',
 'PassDecision',
 'ProposeDecision',
 'SingleSiteMoves',
 'AddDecision',
 'PromotionDecision',
 'RemoveDecision',
 'RotationDecision',
 'RotationDecisionFrequency',
 'TwoSitesMoves',
 'StepDecision',
 'StepDecisionToEmpty',
 'StepDecisionToFriend',
 'StepDecisionToEnemy',
 'SlideDecisi

In [26]:
df_stripped = df_stripped.drop(other_categoricals)

In [27]:
empty_list = []
rows_num = df_stripped.shape[0]

for column in df_stripped.columns:
    null_count = df_stripped.select(column).null_count().to_numpy()[0][0]
    if null_count == rows_num:
        empty_list.append(column)

empty_list 

['Behaviour',
 'StateRepetition',
 'Duration',
 'Complexity',
 'GameOutcome',
 'StateEvaluation',
 'Clarity',
 'Decisiveness',
 'Drama',
 'MoveEvaluation',
 'StateEvaluationDifference',
 'BranchingFactor',
 'DecisionFactor',
 'MoveDistance',
 'PieceNumber',
 'ScoreDifference']

In [28]:
df_stripped = df_stripped.drop(empty_list)

In [29]:
df_stripped.write_csv('train_stripped.csv')

In [30]:
df_stripped = df_stripped.with_columns(
    pl.concat_str(
        [
            pl.col('agent1'),
            pl.col('agent2'),
        ],
        separator='_'
    ).alias('rival'),
)


In [31]:
df_stripped = df_stripped.drop(['agent1', 'agent2', 'GameRulesetName'])

In [32]:
df_stripped.write_csv('train_stripped.csv')

In [33]:
df_stripped

Id,NumColumns,NumRows,NumCorners,NumDirections,NumOrthogonalDirections,NumDiagonalDirections,NumAdjacentDirections,NumOffDiagonalDirections,NumOuterSites,NumInnerSites,NumLayers,NumEdges,NumCells,NumVertices,NumPerimeterSites,NumTopSites,NumBottomSites,NumRightSites,NumLeftSites,NumCentreSites,NumConvexCorners,NumConcaveCorners,NumContainers,NumPlayableSites,NumComponentsType,NumComponentsTypePerPlayer,NumDice,NumStartComponentsHand,NumStartComponents,NumStartComponentsHandPerPlayer,NumStartComponentsPerPlayer,PassDecisionFrequency,ProposeDecisionFrequency,AddDecisionFrequency,PromotionDecisionFrequency,RemoveDecisionFrequency,…,DecisionFactorMaxIncrease,DecisionFactorMaxDecrease,MoveDistanceAverage,MoveDistanceMedian,MoveDistanceMaximum,MoveDistanceVariance,MoveDistanceChangeAverage,MoveDistanceChangeSign,MoveDistanceChangeLineBestFit,MoveDistanceChangeNumTimes,MoveDistanceMaxIncrease,MoveDistanceMaxDecrease,PieceNumberAverage,PieceNumberMedian,PieceNumberMaximum,PieceNumberVariance,PieceNumberChangeAverage,PieceNumberChangeSign,PieceNumberChangeLineBestFit,PieceNumberChangeNumTimes,PieceNumberMaxIncrease,PieceNumberMaxDecrease,ScoreDifferenceAverage,ScoreDifferenceMedian,ScoreDifferenceMaximum,ScoreDifferenceVariance,ScoreDifferenceChangeAverage,ScoreDifferenceChangeSign,ScoreDifferenceChangeLineBestFit,ScoreDifferenceMaxIncrease,ScoreDifferenceMaxDecrease,PlayoutsPerSecond,MovesPerSecond,EnglishRules,LudRules,utility_agent1,rival
i64,i64,i64,i64,f64,f64,f64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,f64,str
0,8,8,6,5.0,5.0,0.0,5.0,0.0,21,27,1,120,73,48,21,4,5,1,1,3,6,0,1,48,3,1.0,0,0,0,0.0,0.0,0.01,0.0,0.99,0.0,0.0,…,37.6,-38.58,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.86,24.45,45.2,169.31,0.72,0.72,0.7,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,298.07,18877.17,"""Goal: Connect all three edge c…","""(game ""00'Y'"" (players 2) (equ…",-0.466667,"""MCTS-ProgressiveHistory-0.1-MA…"
1,8,8,6,5.0,5.0,0.0,5.0,0.0,21,27,1,120,73,48,21,4,5,1,1,3,6,0,1,48,3,1.0,0,0,0,0.0,0.0,0.01,0.0,0.99,0.0,0.0,…,37.6,-38.58,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.86,24.45,45.2,169.31,0.72,0.72,0.7,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,298.07,18877.17,"""Goal: Connect all three edge c…","""(game ""00'Y'"" (players 2) (equ…",-0.333333,"""MCTS-ProgressiveHistory-0.1-MA…"
2,8,8,6,5.0,5.0,0.0,5.0,0.0,21,27,1,120,73,48,21,4,5,1,1,3,6,0,1,48,3,1.0,0,0,0,0.0,0.0,0.01,0.0,0.99,0.0,0.0,…,37.6,-38.58,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.86,24.45,45.2,169.31,0.72,0.72,0.7,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,298.07,18877.17,"""Goal: Connect all three edge c…","""(game ""00'Y'"" (players 2) (equ…",-0.066667,"""MCTS-ProgressiveHistory-0.1-MA…"
3,8,8,6,5.0,5.0,0.0,5.0,0.0,21,27,1,120,73,48,21,4,5,1,1,3,6,0,1,48,3,1.0,0,0,0,0.0,0.0,0.01,0.0,0.99,0.0,0.0,…,37.6,-38.58,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.86,24.45,45.2,169.31,0.72,0.72,0.7,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,298.07,18877.17,"""Goal: Connect all three edge c…","""(game ""00'Y'"" (players 2) (equ…",-0.333333,"""MCTS-ProgressiveHistory-0.1-MA…"
4,8,8,6,5.0,5.0,0.0,5.0,0.0,21,27,1,120,73,48,21,4,5,1,1,3,6,0,1,48,3,1.0,0,0,0,0.0,0.0,0.01,0.0,0.99,0.0,0.0,…,37.6,-38.58,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,23.86,24.45,45.2,169.31,0.72,0.72,0.7,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,298.07,18877.17,"""Goal: Connect all three edge c…","""(game ""00'Y'"" (players 2) (equ…",-0.333333,"""MCTS-ProgressiveHistory-0.1-MA…"
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
233229,7,7,4,6.37,3.43,2.94,6.37,0.0,24,25,1,112,49,64,24,7,7,7,7,1,4,0,1,49,2,1.0,0,0,48,0.0,24.0,0.01,0.0,0.0,0.0,0.0,…,49.85,-49.15,2.08,1.95,6.0,1.59,0.0,0.0,0.0

MCTS-ProgressiveHistory-0.1-MAST-false_MCTS-ProgressiveHistory-0.6-Random200-false

In [34]:
df_pl.with_columns(
    (pl.col('agent1').str.split('-').list.get(1)).alias('sel1'),
    (pl.col('agent1').str.split('-').list.get(2)).alias('expcst1'),
    (pl.col('agent1').str.split('-').list.get(3)).alias('playout1'),
    (pl.col('agent1').str.split('-').list.get(4)).alias('scrbnd1'),
    (pl.col('agent2').str.split('-').list.get(1)).alias('sel2'),
    (pl.col('agent2').str.split('-').list.get(2)).alias('expcst2'),
    (pl.col('agent2').str.split('-').list.get(3)).alias('playout2'),
    (pl.col('agent2').str.split('-').list.get(4)).alias('scrbnd2')
)

Id,GameRulesetName,agent1,agent2,Stochastic,Asymmetric,AsymmetricForces,AsymmetricPiecesType,PlayersWithDirections,Cooperation,Team,Shape,SquareShape,HexShape,TriangleShape,DiamondShape,RectangleShape,SpiralShape,CircleShape,StarShape,SquarePyramidalShape,RegularShape,PolygonShape,Tiling,SquareTiling,HexTiling,TriangleTiling,SemiRegularTiling,MorrisTiling,CircleTiling,ConcentricTiling,SpiralTiling,AlquerqueTiling,MancalaBoard,MancalaStores,MancalaTwoRows,MancalaThreeRows,…,ShowPieceValue,ShowPieceState,Implementation,State,StateType,StackState,PieceState,SiteState,SetSiteState,VisitedSites,Variable,SetVar,RememberValues,ForgetValues,SetPending,InternalCounter,SetInternalCounter,PlayerValue,Efficiency,CopyContext,Then,ForEachPiece,DoLudeme,Trigger,PlayoutsPerSecond,MovesPerSecond,EnglishRules,LudRules,utility_agent1,sel1,expcst1,playout1,scrbnd1,sel2,expcst2,playout2,scrbnd2
i64,str,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,…,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,str,str,f64,str,str,str,str,str,str,str,str
0,"""00Y""","""MCTS-ProgressiveHistory-0.1-MA…","""MCTS-ProgressiveHistory-0.6-Ra…",0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,…,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,1,0,0,1,298.07,18877.17,"""Goal: Connect all three edge c…","""(game ""00'Y'"" (players 2) (equ…",-0.466667,"""ProgressiveHistory""","""0.1""","""MAST""","""false""","""ProgressiveHistory""","""0.6""","""Random200""","""false"""
1,"""00Y""","""MCTS-ProgressiveHistory-0.1-MA…","""MCTS-UCB1GRAVE-0.6-NST-true""",0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,…,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,1,0,0,1,298.07,18877.17,"""Goal: Connect all three edge c…","""(game ""00'Y'"" (players 2) (equ…",-0.333333,"""ProgressiveHistory""","""0.1""","""MAST""","""false""","""UCB1GRAVE""","""0.6""","""NST""","""true"""
2,"""00Y""","""MCTS-ProgressiveHistory-0.1-MA…","""MCTS-UCB1-0.1-NST-false""",0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,…,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,1,0,0,1,298.07,18877.17,"""Goal: Connect all three edge c…","""(game ""00'Y'"" (players 2) (equ…",-0.066667,"""ProgressiveHistory""","""0.1""","""MAST""","""true""","""UCB1""","""0.1""","""NST""","""false"""
3,"""00Y""","""MCTS-ProgressiveHistory-0.1-MA…","""MCTS-UCB1-0.6-NST-false""",0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,…,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,1,0,0,1,298.07,18877.17,"""Goal: Connect all three edge c…","""(game ""00'Y'"" (players 2) (equ…",-0.333333,"""ProgressiveHistory""","""0.1""","""MAST""","""true""","""UCB1""","""0.6""","""NST""","""false"""
4,"""00Y""","""MCTS-ProgressiveHistory-0.1-MA…","""MCTS-UCB1GRAVE-1.41421356237-N…",0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,…,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,1,0,0,1,298.07,18877.17,"""Goal: Connect all three edge c…","""(game ""00'Y'"" (players 2) (equ…",-0.333333,"""ProgressiveHistory""","""0.1""","""MAST""","""true""","""UCB1GRAVE""","""1.41421356237""","""NST""","""false"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
233229,"""Zuz_Mel_7x7""","""MCTS-UCB1Tuned-1.41421356237-N…","""MCTS-ProgressiveHistory-1.4142…",0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,157.52,157174.58,"""7x7 board. 24 pieces per playe…","""(game ""Zuz Mel (7x7)"" (players…",-0.733333,"""UCB1Tuned""","""1.41421356237""","""NST""","""false""","""ProgressiveHistory""","""1.41421356237""","""Random200""","""false"""
233230,"""Zuz_Mel_7x7""","""MCTS-UCB1Tuned-1.41421356237-R…","""MCTS-UCB1-0.6-MAST-false""",0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,…,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [35]:
concepts_df

Id,Name,Description,TypeId,DataTypeId,ComputationTypeId,TaxonomyString,LeafNode,ShowOnWebsite
i64,str,str,i64,i64,i64,str,i64,i64
1,"""Properties""","""General properties of the game…",1,1,1,"""1""",0,0
2,"""Format""","""Format of the game.""",1,1,1,"""1.1""",0,0
3,"""Time""","""Time model.""",1,1,1,"""1.1.1""",0,0
4,"""Discrete""","""Players move at discrete inter…",1,1,1,"""1.1.1.1""",1,1
5,"""Realtime""","""Moves not discrete.""",1,1,1,"""1.1.1.2""",1,1
…,…,…,…,…,…,…,…,…
805,"""SkillTrace""","""Skill Trace.""",7,1,2,"""4.13""",0,1
806,"""SkillTraceScore""","""Skill Trace Score.""",7,4,2,"""4.13.1""",1,1
807,"""SkillTraceTrials""","""Skill Trace Trials.""",7,4,2,"""4.13.2""",1,1
